## **1. Load in data**

In [47]:
# 1. LOAD RAW DATA
import pandas as pd
import numpy as np
from pathlib import Path
import json

data_path = Path("../data/raw_credit_applications.json")
with open(data_path, 'r') as f:
    raw = json.load(f)

df_raw = pd.json_normalize(raw)
print(f"✅ Raw data: {len(df_raw)} rows x {len(df_raw.columns)} cols")
df_raw.head(2)

✅ Raw data: 502 rows x 21 cols


,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.credit_history_months,financials.debt_to_income,financials.savings_balance,decision.loan_approved,decision.rejection_reason,loan_purpose,decision.interest_rate,decision.approved_amount,financials.annual_salary,notes
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,23,0.20,31212,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,51,0.18,17915,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN


✅ Successfully loaded nested JSON to flat DataFrame

📏 Shape: 502 x 21 columns

🔜 Next: Data Quality Audit

## **2. Data quality check**

### 2.1 Dataset overview and data types

In [48]:
# 2.1 DATASET OVERVIEW & DTYPES
print("SHAPE:", df_raw.shape)
print("\nDTYPES (sorted):")
print(df_raw.dtypes.sort_values())

SHAPE: (502, 21)

DTYPES (sorted):
decision.loan_approved                 bool
financials.savings_balance            int64
financials.credit_history_months      int64
decision.approved_amount            float64
decision.interest_rate              float64
financials.debt_to_income           float64
financials.annual_salary            float64
_id                                  object
loan_purpose                         object
decision.rejection_reason            object
financials.annual_income             object
applicant_info.date_of_birth         object
applicant_info.gender                object
applicant_info.ip_address            object
applicant_info.ssn                   object
applicant_info.email                 object
applicant_info.full_name             object
processing_timestamp                 object
spending_behavior                    object
applicant_info.zip_code              object
notes                                object
dtype: object


### 2.1 FINDINGS: Dataset Overview & Dtypes

**Dataset info:**
- Shape: **502 rows × 21 columns** ✅ Normal size

**Critical Data Type Issues (CONSISTENCY):**
- ❌ `financials.annual_income`: **object** (should be numeric)  
  *Cleaning:* `pd.to_numeric(errors='coerce')`

**Other observations:**
- 6 numeric columns OK (int64/float64)
- 15 object columns (some should be numeric/date)
- `spending_behavior`: object → likely list → handle separately

**DQ Impact:** High – blocks numeric analysis [file:3]

### 2.2 Missing values

In [49]:
# 2.2 MISSING VALUES ANALYSIS
missing = df_raw.isna().sum().sort_values(ascending=False)
missing_pct = (missing / len(df_raw)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct.round(1)
}).query('missing_pct > 0')
print(missing_df.head(10))

                           missing_count  missing_pct
notes                                500         99.6
financials.annual_salary             497         99.0
loan_purpose                         452         90.0
processing_timestamp                 440         87.6
decision.rejection_reason            292         58.2
decision.interest_rate               210         41.8
decision.approved_amount             210         41.8
applicant_info.ssn                     5          1.0
applicant_info.ip_address              5          1.0
financials.annual_income               5          1.0


### 2.2 FINDINGS: Missing Values Analysis

**Top Missing (COMPLETENESS dimension):**

| Column                    | Missing Count | Missing % |
|---------------------------|---------------|-----------|
| notes                     | **500**       | **99.6%** |
| financials.annual_salary  | 497           | **99.0%** |
| loan_purpose              | 452           | **90.0%** |
| processing_timestamp      | 440           | **87.6%** |
| decision.rejection_reason | 292           | **58.2%** |
| decision.interest_rate    | 210           | **41.8%** |
| decision.approved_amount  | 210           | **41.8%** |
| applicant_info.ssn        | 5             | **1.0%**  |
| applicant_info.ip_address | 5             | **1.0%**  |
| financials.annual_income  | 5             | **1.0%**  |

**Business interpretation:**
- ✅ `rejection_reason`, `interest_rate`, `approved_amount`: expected missing (only for rejected/approved)
- ❌ `notes`, `annual_salary`, `loan_purpose`: **severe completeness issues**
  
**Cleaning steps:**
1. Drop `notes` (99.6% missing = useless)
2. Flag `annual_salary` as unreliable (99% missing)
3. Impute or drop high-missing rows for `loan_purpose`

**DQ Impact:** Medium-High – affects analysis reliability [file:3]


### 2.3 Duplicates

In [51]:
# 2.3 DUPLICATES CHECK (safe for list columns)
print("Checking duplicates (excluding list columns)...")

# Exclude unhashable columns (lists)
hashable_cols = []
for col in df_raw.columns:
    try:
        if df_raw[col].dropna().empty:
            hashable_cols.append(col)
        elif not any(isinstance(x, list) for x in df_raw[col].dropna().head()):
            hashable_cols.append(col)
    except:
        pass

df_check = df_raw[hashable_cols].copy()
dup_count = df_check.duplicated().sum()
dup_pct = round((dup_count / len(df_check)) * 100, 2)

print(f"Duplicate rows: {dup_count} ({dup_pct}%)")
if dup_count > 0:
    print("\nSample duplicates:")
    print(df_check[df_check.duplicated()].head(2))
else:
    print("✅ No duplicates found")

Checking duplicates (excluding list columns)...
Duplicate rows: 0 (0.0%)
✅ No duplicates found


### 2.3 FINDINGS: Duplicates Check

**Results:**
- Duplicate rows: **0 (0.0%)** ✅ **Clean!**

**Notes:**
- Checked excluding unhashable list columns (`spending_behavior`)
- No exact row duplicates found

**DQ dimension:** ACCURACY ✅ **No action needed**

**Business key check recommendation:** 
- Verify `_id` uniqueness: `df_raw['_id'].nunique() == len(df_raw)`


### 2.4 Numeric columns overview

In [57]:
# 2.4 NUMERIC COLUMNS OVERVIEW
num_cols = df_raw.select_dtypes(include=['number']).columns.tolist()

print("NUMERIC SUMMARY")
print("-" * 60)
print(f"{'Column':<25} {'Min':>8} {'Max':>8} {'Mean':>8} {'Miss%':>6} {'Negatives'}")

for col in num_cols:
    stats = df_raw[col].describe()
    min_val = stats['min']
    max_val = stats['max']
    mean_val = stats['mean']
    miss_pct = round(df_raw[col].isna().mean()*100, 1)
    negatives = (df_raw[col] < 0).sum()
    
    neg_flag = f"{negatives} ({negatives/len(df_raw)*100:.1f}%)" if negatives > 0 else "0"
    
    print(f"{col:<25} {min_val:>8.1f} {max_val:>8.1f} {mean_val:>8.1f} {miss_pct:>6}% {neg_flag}")

NUMERIC SUMMARY
------------------------------------------------------------
Column                         Min      Max     Mean  Miss% Negatives
financials.credit_history_months    -10.0    133.0     50.4    0.0% 2 (0.4%)
financials.debt_to_income      0.1      1.9      0.2    0.0% 0
financials.savings_balance  -5000.0  88078.0  29493.5    0.0% 1 (0.2%)
decision.interest_rate         2.5      6.5      4.6   41.8% 0
decision.approved_amount   15000.0  80000.0  47845.9   41.8% 0
financials.annual_salary   45000.0  94000.0  69200.0   99.0% 0


### 2.4 FINDINGS: Numeric Analysis (VALIDITY dimension)

**Key Observations:**
- **6 numeric columns detected** (vs 7 expected) → `annual_income` object as is
- Range looks reasonable except...

**Critical Issues:**
1. **`financials.credit_history_months`**: **Min < 0** (2 records, 0.4%) ❌ **Invalid**
   - *Cleaning:* `clip(lower=0)` or `set to NaN`
   
2. **`financials.savings_balance`**: **Min < 0** (1 record, 0.2%) ⚠️ Borderline (overdraft?)
   - *Cleaning:* Document as business valid

**Cleaning steps summary:**

- df['credit_history_months'] = df['credit_history_months'].clip(lower=0)

- Flag overdraft records

- Convert annual_income to numeric (from 2.1)

### 2.5 Categorical column overview

In [64]:
# 2.5 CATEGORICAL OVERVIEW (safe)
print("ALL CATEGORICAL - Uniques & Top values")
print("="*60)

for col in df_raw.columns:
    if df_raw[col].dtype == 'object':
        print(f"\n{col}:")
        try:
            vc = df_raw[col].value_counts(dropna=False).head(4)
            print(vc)
        except:
            print("  [SKIPPED - list column]")

ALL CATEGORICAL - Uniques & Top values

_id:
_id
app_001    2
app_042    2
app_453    1
app_177    1
Name: count, dtype: int64

spending_behavior:
spending_behavior
[{'category': 'Insurance', 'amount': 171}]    2
[{'category': 'Utilities', 'amount': 566}]    2
[{'category': 'Utilities', 'amount': 786}]    2
[{'category': 'Fitness', 'amount': 98}]       2
Name: count, dtype: int64

processing_timestamp:
processing_timestamp
NaN                     440
2024-01-15T00:00:00Z     59
2025-12-01T00:00:00Z      1
2026-03-15T00:00:00Z      1
Name: count, dtype: int64

applicant_info.full_name:
applicant_info.full_name
Susan Flores    3
Rachel King     2
George Clark    2
James Rivera    2
Name: count, dtype: int64

applicant_info.email:
applicant_info.email
                               7
joseph.lopez1@gmail.com        2
stephanie.nguyen47@mail.com    2
nicholas.king46@outlook.com    1
Name: count, dtype: int64

applicant_info.ssn:
applicant_info.ssn
NaN            5
780-24-9300    2
652-70-55

## 2.5 FINDINGS: Categorical Analysis (CONSISTENCY + ACCURACY)

**Top Issues:**

1. **`applicant_info.gender`**: **Severe mixed coding** ❌  
   - Male: 195, Female: 193, **F: 58**, *(M is likely present)*  
   - *Issue:* 3-4+ formats → unusable for bias analysis  
   - *Fix:* `replace({'F': 'Female', 'M': 'Male'}).str.capitalize()`

2. **`_id`**: **PRIMARY KEY VIOLATION** ❌  
   - app_001: **2x**, app_042: **2x**  
   - *Issue:* Duplicated application IDs  
   - *Fix:* `drop_duplicates(subset='_id', keep='first')`

3. **`applicant_info.full_name`**: **Person-level duplicates** ⚠️  
   - Susan Flores: **3x** occurrences  
   - *Issue:* Same individual multiple apps?  

4. **`financials.annual_income`**: **String numbers** (2.1 confirm)  
   - Top: "79000"(11x), "100000"(10x)  
   - *Fix:* `pd.to_numeric(errors='coerce')`

5. **`applicant_info.email`**: **Empty PII** ⚠️  
   - **7 completely blank** → privacy/governance flag

**Priority Summary:**
| Priority | Column | Issue Type | Fix |
|----------|--------|------------|-----|
| **CRITICAL** | _id | Duplicate PK | drop_duplicates() |
| **HIGH** | gender | Mixed coding | replace mapping |
| MEDIUM | annual_income | Type mismatch | to_numeric() |
| LOW | full_name | Potential duplicates | business decision |

**Business Impact:** Gender unusable for **bias analysis**; trust issues with IDs [file:3]

### 2.6 Date Consistency Check


In [63]:
# 2.6 DATE FORMAT CONSISTENCY CHECK
date_col = 'applicant_info.date_of_birth'

print("DATE OF BIRTH FORMATS ANALYSIS")
print("-" * 50)

# Sample non-null dates
dates = df_raw[date_col].dropna().unique()[:20]  # Top 20 unique
print(f"Unique date formats (sample {len(dates)}):")
for d in dates:
    print(f"  '{d}' → len: {len(str(d))}, format: {pd.to_datetime(d, errors='coerce', infer_datetime_format=True)}")

# Format patterns
date_patterns = df_raw[date_col].dropna().str.len().value_counts().sort_index()
print(f"\nDate string lengths: {dict(date_patterns)}")

# Try parsing
df_raw['date_parsed'] = pd.to_datetime(df_raw[date_col], errors='coerce')
invalid_dates = df_raw['date_parsed'].isna().sum()
print(f"\nInvalid dates: {invalid_dates} ({invalid_dates/len(df_raw)*100:.1f}%)")

DATE OF BIRTH FORMATS ANALYSIS
--------------------------------------------------
Unique date formats (sample 20):
  '2001-03-09' → len: 10, format: 2001-03-09 00:00:00
  '1992-03-31' → len: 10, format: 1992-03-31 00:00:00
  '1989-10-24' → len: 10, format: 1989-10-24 00:00:00
  '1983-04-25' → len: 10, format: 1983-04-25 00:00:00
  '1999-05-21' → len: 10, format: 1999-05-21 00:00:00
  '14/02/1982' → len: 10, format: 1982-02-14 00:00:00
  '28/01/1990' → len: 10, format: 1990-01-28 00:00:00
  '1991-10-11' → len: 10, format: 1991-10-11 00:00:00
  '1990-05-04' → len: 10, format: 1990-05-04 00:00:00
  '1989-10-10' → len: 10, format: 1989-10-10 00:00:00
  '1970-10-01' → len: 10, format: 1970-10-01 00:00:00
  '01/12/1978' → len: 10, format: 1978-01-12 00:00:00
  '1989-06-13' → len: 10, format: 1989-06-13 00:00:00
  '1976-04-19' → len: 10, format: 1976-04-19 00:00:00
  '1990/07/26' → len: 10, format: 1990-07-26 00:00:00
  '1992-06-22' → len: 10, format: 1992-06-22 00:00:00
  '1997-03-23' → len:

C:\Users\mikis\AppData\Local\Temp\ipykernel_28180\3466476320.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  print(f"  '{d}' → len: {len(str(d))}, format: {pd.to_datetime(d, errors='coerce', infer_datetime_format=True)}")
C:\Users\mikis\AppData\Local\Temp\ipykernel_28180\3466476320.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  print(f"  '{d}' → len: {len(str(d))}, format: {pd.to_datetime(d, errors='coerce', infer_datetime_format=True)}")


### 2.6 Date Format Analysis

**Key Stats:**
- 497 unique dates (len=10), 4 empty → **32.3% parse failure** ❌

**Mixed Formats:**
| Format | Example | Parses? |
|--------|---------|---------|
| yyyy-mm-dd | 2001-03-09 | ✅ |
| dd/mm/yyyy | 14/02/1982 | ✅ |
| yyyy/mm/dd | 1990/07/26 | ✅ |

**Problem:** **3 international format** → inconsistent parsing (Pandas warning)

**DQ Category:** **CONSISTENCY** (critical for age/bias analysis)

**Fix Plan:**
- pd.to_datetime(dayfirst=True, errors='coerce')
- Convert all to ISO format (YYYY-MM-DD)
- 32.3% NaT → impute or drop (investigate)

**Business impact:** Blocks age/bias analysis

- gender (applicant_info.gender	): male / m
- date (applicant_info.date_of_birth): yyyy/mm/dd ; dd/mm/yyyy ; yyyy-mm-dd
- zip code (applicant_info.zip_code): num		

In [19]:
df['applicant_info.gender'].value_counts()

applicant_info.gender
Male      195
Female    193
F          58
M          53
            2
Name: count, dtype: int64

if 3rd char / dd mm yyy else yyyy mm dd

In [20]:
# Normalize gender column to a single format (all lowercase)
df['applicant_info.gender'] = df['applicant_info.gender'].str.lower()

# Verify the normalization
df['applicant_info.gender'].value_counts()

applicant_info.gender
male      195
female    193
f          58
m          53
            2
Name: count, dtype: int64